In [334]:
%store -r tokens

In [335]:
tokens = tokens

In [336]:
print(tokens)

['#include', '<', 'iostream', '>', 'using', 'namespace', 'std', ';', 'int', 'main', '(', ')', '{', 'int', 'first_number', '=', '-', '52', ',', 'sum', ';', 'double', 'a', '=', 'first_number', ';', 'int', 'd', '=', 'a', '+', 'a', ';', 'bool', 'b', '=', 'true', ';', 'string', 's', '=', '"anand"', ';', 'char', 'a', '=', "'a'", ';', 'sum', '=', 'first_number', ';', 'int', 'i', ';', 'for', '(', 'int', 'i', '=', '2', ',', 'j', '=', '5', ';', 'i', '<', '5', ';', 'i', '=', 'i', '+', '1', ')', '{', 'j', '=', '6', ';', '}', 'j', '=', '5', ';', 'if', '(', 'true', ')', '{', 'int', 'my_num', '=', 'first_number', '+', 'a', ';', '}', 'else', '{', 'int', 'b', '=', '2', ';', '}', '}']


In [337]:
global_symbol_table = {}

# symbol table structure {'a':['int'], 'func1' : ['int', {'a': 'int'}], 'func2' : ['void', {}]}

### Ignore Preprocessors

In [338]:
def ignorePreprocessor(tokens, index):
    while(tokens[index] != ';'):
        index+=1
    return index+1

In [339]:
datatype=["int","double","float","char","string", "void", "bool"]

integer_dtypes = ["int"]
decimal_dtypes = ["float", "double"]
boolean_dtypes = ["bool"]

In [340]:
rval_operators=['+', '-', '*', '/', '&&', '||', '&', '|', '<', '>', '^', '==', '>=', '<=']
lval_operators=['+=', '-=', '=', '*=', '/=']
unary=['!', '-', '+']

# Check identifier and constants

In [341]:
# returns isidentifier
def isIdentifier(s):
    if(not((s[0]=='_')or(s[0]>='a' and s[0]<='z')or(s[0]>='A' and s[0]<='Z'))):
        return False
    for i in range(1,len(s)):
        if(not((s[i]=='_')or(s[i]>='a' and s[i]<='z')or(s[i]>='A' and s[i]<='Z')or(s[i]>='0' and s[i]<='9'))):
            return False
    return True

# return isConst, dtype
def isConst(s):
    dtype = ""

    if(s == "true" or s == "false"):
        return True, "bool"
    
    if(s[0] == '"' and s[-1] == '"'):
        return True, "string"
    
    if(s[0] == "'" and s[-1] == "'" and len(s) == 3):
        return True, "char"

    try:
        int(s)
        return True, "int"
    except ValueError:
        try:
            float(s)
            return True, "float"
        except ValueError:
            return False, dtype


# CheckLval

In [342]:
# returns index, err, dtype, islval

def checkLval(tokens, index, symbol_table):
    print("inside checkLval", index,tokens[index])

    dtype = ""
    res, const_dtype = isConst(tokens[index])
    
    if(res):
        index += 1
        return index, None, const_dtype, False
    
    if(isIdentifier(tokens[index])):
        # check if present in symbol table and get dtype from symbol table
        ident = tokens[index]

        if(ident not in symbol_table):
            return index, "variable not declared", dtype, False # deepak lval return check
#             return index, "variable not declared"
        index += 1

        # if the ident is a function
        if(tokens[index] == '('):
            index += 1
            params = []
            while(tokens[index] != ')'):
                if(not isIdentifier(tokens[index])):
                    return index, "invalid identifier", dtype, False
                index+=1
                # don't increement index here
                
                id = tokens[index]
                if(id in symbol_table):
                    params.append(symbol_table[id][0])
                if(tokens[index] != ',' and tokens[index] != ')'):
                    return index, "unexpected token inside function arguments", dtype, False
                index += 1
            
            
                if(len(symbol_table[ident]) < 2):
                    return index, "identifier is not callable", dtype, False
                
                # check this after existing from while loop and take care of , as well
                
                params_list = list(symbol_table[ident][1].values())
                if(params_list != params):
                    return index, "parameter list of function doesn't match", dtype, False
                
            return index, None, symbol_table[ident][0], False
   
        # if not a function call
        return index, None, symbol_table[ident][0], True
        
    if(tokens[index] in unary):
        op = tokens[index]
        index += 1
        index, err, data_type, sub_lval = checkLval(tokens=tokens, index=index, symbol_table=symbol_table)
        print(f"exited lval with {index}, token:{tokens[index]}, data_type: {data_type}")
        if(op == '!'):
            return index, err, 'bool', False
        else: # operator = '-' or '+'
            if((data_type not in integer_dtypes) and (data_type not in decimal_dtypes) and (data_type not in boolean_dtypes)):
                return index, f"operator {op} on non numerical data type", dtype, False
            return index, err, data_type, False

    if(tokens[index] == '('):
        index += 1
        index, err, data_type,ig_bool = checkLval(tokens=tokens, index=index, symbol_table=symbol_table) # deepak lval return check
        if(err):
            return index, err, data_type, False
        
        if(tokens[index] != ')'):
            return index, "unmatched braces", data_type, False
        index += 1 # matched braces
        return index, err, data_type, False

    return index, "unexpected token inside lvalue", dtype, False


# CheckExpr

In [343]:
# It doesn't parses semicolon
# returns index, err, dtype

def checkExpr(tokens, index, symbol_table, islval=True):
    print("inside checkExpr", index,tokens[index])
    dtype = ""
    index, err, ldtype, islvalue = checkLval(tokens, index, symbol_table)
    rdtype = ""
    print(f"exited checkLval {index}, {tokens[index]}, ldtype: {ldtype}")
    if(err):
        return index, err, dtype

    if(tokens[index] in lval_operators):
        if(islval and islvalue):
            index += 1
            index, err, rdtype = checkExpr(tokens=tokens, index=index, symbol_table=symbol_table, islval=True)
            print(f"exited checkExpr {index}, {tokens[index]}, err: {err}")
        else:
            return index, "lval operators cannot be applied on rval", dtype
    
    elif(tokens[index] in rval_operators):
        index += 1
        index, err, rdtype = checkExpr(tokens=tokens, index=index, symbol_table=symbol_table, islval=False)
        print(f"exited checkExpr {index}, {tokens[index]}, err: {err}")

    else:
        return index, None, ldtype # if there is no operator after an lvalue

    if(err): # error already then return error
        return index, err, dtype # deepak return check
    
    print(f"checking for semicolon and datatype after rval op, ldtype: {ldtype}, rdtype: {rdtype}")
    # decide dtype on basis of ldtype and rdtype
    if(ldtype == "void" or rdtype == "void"):
        print("ldtype in void type")
        return index, "no operation allowed on void datatype", dtype

    if(ldtype == "string"):
        print("ldtype in string type")
        if(not (rdtype == "char" or rdtype == "string")):
            return index, "mismatched types", dtype
        return index, None, "string"
    
    if(ldtype == "char"):
        print("ldtype in char type")
        if(rdtype == "string"):
            return index, "string cannot be appended to a char", dtype
        
        if(rdtype in boolean_dtypes):
            return index, None, "int"

        return index, None, rdtype

    if(ldtype in integer_dtypes):
        print("ldtype in integer type")
        if(rdtype == "string"):
            return index, "mismatched types", dtype

        if(rdtype == "char"):
            return index, None, ldtype

        if(rdtype in boolean_dtypes):
            return index, None, ldtype

        if(rdtype in decimal_dtypes):
            return index, None, rdtype

        if(rdtype in integer_dtypes):
            if(integer_dtypes.index(rdtype) >= integer_dtypes.index(ldtype)):
                return index,None, rdtype
            else:
                return index,None, ldtype

    if(ldtype in decimal_dtypes):
        print("ldtype in decimal type")
        if(rdtype == "string"):
            return index, "mismatched types", dtype

        if(rdtype == "char"):
            return index, None, ldtype

        if(rdtype in boolean_dtypes):
            return index, None, ldtype

        if(rdtype in integer_dtypes):
            return index, None, ldtype

        if(rdtype in decimal_dtypes):
            if(decimal_dtypes.index(rdtype) >= decimal_dtypes.index(ldtype)):
                return index,None, rdtype
            else:
                return index,None, ldtype

    if(ldtype in boolean_dtypes):
        print("ldtype in boolean type")
        if(rdtype == "string"):
            return index, "mismatched types", dtype

        if(rdtype == "char"):
            return index, None, "int"
        
        return index, None, rdtype
    
    return index, "datatype invalid", dtype


# Check function declaration and variable definition ( Scoped )

In [344]:
# returns index, err, symbol_table
def checkFuncDecOrValDef(tokens, index, symbol_table):
    if(tokens[index] in datatype):
        index, err, dtype, ident = checkVarHeader(tokens=tokens, index=index)

    if(err):
        return index, err, symbol_table

    symbol_table[ident] = [dtype]

    
    # variable decl
    if(tokens[index] == ";"):
        if(dtype == "void"):
            return index, "wrong datatype for variable declation", symbol_table
        return index+1, None, symbol_table
    
        # variable def
    if(tokens[index] == "="):
        if(dtype == "void"):
            return index, "wrong datatype for variable declation", symbol_table
        index += 1
        index,err,expr_dtype =  checkExpr(tokens, index, symbol_table)
        print(f"exited checkExpr inside var def fun dec {index}, {tokens[index]}, err: {err}")
        if(err):
            return index, err, symbol_table
        if(tokens[index] == ","):
            while(tokens[index] == ','):
                index += 1
                if(not isIdentifier(tokens[index])):
                    return index, "expected identifier"
                symbol_table[tokens[index]] = [dtype]
                index += 1

                if(tokens[index] == '='):
                    index += 1
                    index, err, expr_dtype = checkExpr(tokens=tokens, index=index, symbol_table=symbol_table)
                    if(err):
                        return index, err, symbol_table
                    
        if(tokens[index] != ';'):
            return index, "expected ; after an expression", symbol_table
        index +=1
        return index,None, symbol_table
    
    if(tokens[index] == ","):
        while(tokens[index] == ','):
            index += 1
            if(not isIdentifier(tokens[index])):
                return index, "expected identifier"
            symbol_table[tokens[index]] = [dtype]
            index += 1

            if(tokens[index] == '='):
                index += 1
                index, err, expr_dtype = checkExpr(tokens=tokens, index=index, symbol_table=symbol_table)
                if(err):
                    return index, err, symbol_table
                
        if(tokens[index] != ";"):
            return index, "expected ; ", symbol_table
        index += 1
        return index, None, symbol_table

    index, err, parameters = checkParametersList(tokens=tokens, index=index)

    if(err):
        return index, err, symbol_table
    
    # add function to the symbol table
    symbol_table[ident] = [dtype, parameters]

    if(tokens[index] != ";"):
        return index, "Invalid function declaration", symbol_table
    
    return index+1, None, symbol_table

# checkForVariableDecl

In [345]:
# returns index, err, symbol_table
def checkForVariableDecl(tokens, index, symbol_table):
    dtype = ""
    isdec = False

    if(tokens[index] == ';'):
        return index, None, symbol_table
    
    if(tokens[index] in datatype):
        dtype = tokens[index]
        if(dtype == "void"):
            return index, "cannot be void", symbol_table
        isdec = True
        index += 1

    if(not isIdentifier(tokens[index])):
        return index, "expected an identifier", symbol_table
    
    ident = tokens[index]
    if(isdec):
        symbol_table[ident] = [dtype]
    else:
        if(ident not in symbol_table):
            return index, "variable undeclared", symbol_table
    index += 1

    if(tokens[index] == "="):
        if(dtype == "void"):
            return index, "wrong datatype for variable declation"
        index += 1
        print("checking expr for: ",index,tokens[index])
        index,err,ig_dtype = checkExpr(tokens=tokens, index=index, symbol_table=symbol_table) # deepak return check
        if(err):
            return index, err, symbol_table

    if(tokens[index] == ","):
        while(tokens[index] == ','):
            index += 1
            if(not isIdentifier(tokens[index])):
                return index, "expected identifier"
            ident = tokens[index]

            if(isdec):
                symbol_table[ident] = [dtype]
            else:
                if(ident not in symbol_table):
                    return index, "variable undeclared", symbol_table
            
            index += 1

            if(tokens[index] == '='):
                index += 1
                index, err, expr_dtype = checkExpr(tokens=tokens, index=index, symbol_table=symbol_table)
                if(err):
                    return index, err, symbol_table
                
    return index, None, symbol_table


# Check Scope

In [346]:
# returns index, err
def checkScope(tokens, index, symbol_table):

    if(tokens[index] != '{'):
        return index, f"Expected {{ but found {tokens[index]}"
    index+=1

    local_symbol_table = {}
    local_symbol_table.update(symbol_table)

    while(tokens[index] != "}"):
        print(f"processing {index} with token: {tokens[index]}")

        if(tokens[index] in datatype):
            print("it is a func declaration or var def")
            index, err, sym_tab = checkFuncDecOrValDef(tokens=tokens, index=index, symbol_table=local_symbol_table)
            if(err):
                return index, err
            local_symbol_table.update(sym_tab)
            print("processed func declaration or var def")
            continue
            
        
        elif(tokens[index] == "if"):
            print("it is a if statement")
            index += 1
            if(tokens[index] != "("):
                return index, f"Expected ( but found {tokens[index]}"
            index += 1
            ig_idx,ig_err,ig_dtype = checkExpr(tokens=tokens, index=index, symbol_table=local_symbol_table) # deepak return check
            index, err = ig_idx,ig_err # deepak return check
            if(err):
                return index, err
            if(tokens[index] != ")"):
                return index, f"Expected ) but found {tokens[index]}"

            index +=1

            index, err = checkScope(tokens=tokens, index=index, symbol_table=local_symbol_table)
            if(err):
                return index, err
            
            #update else if

            if(tokens[index] == "else"):
                index += 1
                index, err = checkScope(tokens=tokens, index=index, symbol_table=local_symbol_table)
                if(err):
                    return index, err
            print("processed if statement")
            continue

        elif(tokens[index] == "while"):
            print("it is a while statement")
            index += 1

            if(tokens[index] != "("):
                return index, f"Expected ( but found {tokens[index]}"
            index+=1
            ig_idx,ig_err,ig_dtype = checkExpr(tokens=tokens, index=index) # deepak return check
            index, err = ig_idx,ig_err # deepak return check
            
            if(err):
                return index, err

            if(tokens[index] != ")"):
                return index, f"Expected ) but found {tokens[index]}"
            index +=1

            index, err = checkScope(tokens=tokens, index=index, symbol_table=local_symbol_table)

            if(err):
                return index, err
            print("processed while statement")
            continue

        elif(tokens[index] == "do"):
            print("it is a do while statement")
            index += 1

            index, err = checkScope(tokens=tokens, index=index, symbol_table=local_symbol_table)
            if(err):
                return index, err

            if(tokens[index] != "while"):
                return index, f"Expected while but found {tokens[index]}"
            index += 1

            if(tokens[index] != "("):
                return index, f"Expected ( but found {tokens[index]}"
            index += 1
            ig_idx,ig_err,ig_dtype = checkExpr(tokens=tokens, index=index, symbol_table=local_symbol_table) # deepak return check
            index, err = ig_idx,ig_err # deepak return check
            
            if(err):
                return index, err

            if(tokens[index] != ")"):
                return index, f"Expected ) but found {tokens[index]}"
            index +=1

            if(tokens[index] != ";"):
                return index, f"Expected ) but found {tokens[index]}"
            index +=1
            print("processed do while statement")
            continue

        elif(tokens[index] == "for"):
            print("it is a for statement")
            index += 1
            if(tokens[index] != "("):
                return index, "expected ("
            index += 1
            index, err, symbol_table = checkForVariableDecl(tokens=tokens, index=index, symbol_table=symbol_table)
            print(f"exit from variable declaration {symbol_table}, err: {err}, token: {tokens[index]}")
            if(err):
                return index, err
            if(tokens[index] != ";"):
                return index, "expected ;"
            index += 1
            index, err, expr1_dtype = checkExpr(tokens=tokens, index=index, symbol_table=symbol_table)
            if(err):
                return index, err
            if(tokens[index] != ";"):
                return index, "expected ;"
            index += 1
            index, err, expr1_dtype = checkExpr(tokens=tokens, index=index, symbol_table=symbol_table)
            if(err):
                return index, err
            if(tokens[index]!= ')'):
                return index, "expected )"
            index += 1
            index, err = checkScope(tokens=tokens, index=index, symbol_table=symbol_table)
            if(err):
                return index, err
            print("processed for statement")
            continue

        else:
            print("it is a expression")
            ig_idx,ig_err,ig_dtype = checkExpr(tokens=tokens, index=index, symbol_table=local_symbol_table) # deepak return check
            index, err = ig_idx,ig_err # deepak return check
            print(f"exited checkExpr {index}, {tokens[index]}")
            if(err):
                return index, err
            if(tokens[index] != ';'):
                return index, "unexpected token after an expression"
            index +=1
            print("processed expression")
            continue
    
    return index+1,None
        

# Check Var Header

In [347]:
# returns index, err, dtype, ident

def checkVarHeader(tokens, index):
    # check construct of type <dtype> <ident>
    dtype=""
    ident=""

    if(tokens[index] not in datatype):
        return index, "invalid data type", dtype, ident
    else:
        dtype = tokens[index]
    index += 1

    # check for identifier
    if(not isIdentifier(tokens[index])):
        return index, "invalid identifier name", dtype, ident
    else:
        ident = tokens[index]
        
    return index+1, None, dtype, ident

# Check parameter list

In [348]:
# returns index, err, parameters

def checkParametersList(tokens, index):
    parameters = {}
    if(tokens[index] != '('):
        return index, "missing opening parenthesis for a function", parameters
    
    index += 1
    while(tokens[index] != ')'):
        index, err, dtype, ident = checkVarHeader(tokens, index)
        parameters[ident] = dtype
        if(err):
            return index, err, parameters
        if(tokens[index] == ','):
            index += 1
        else:
            break
    
    if(tokens[index] != ')'):
        return index, "missing closing parenthesis for a function", parameters
    
    return index+1, None, parameters


# check function definition and variable definition

In [349]:
# returns index, err
def checkFuncDefOrVarDef(tokens, index):
    index, err, dtype, ident = checkVarHeader(tokens=tokens, index=index)

    if(err):
        return index, err
    
    # check for multiple declaration
    
    global_symbol_table[ident] = [dtype]
    
    # variable decl
    if(tokens[index] == ";"):
        if(dtype == "void"):
            return index, "wrong datatype for variable declation"
        return index+1, None
    
    # variable def
    if(tokens[index] == "="):
        if(dtype == "void"):
            return index, "wrong datatype for variable declation"
        index += 1
        print("checking expr for: ",index,tokens[index])
        ig_idx,ig_err,ig_dtype = checkExpr(tokens=tokens, index=index, symbol_table=global_symbol_table) # deepak return check
        return ig_idx,ig_err # deepak return check
#         return checkExpr(tokens, index, global_symbol_table)

    if(tokens[index] == ","):
        while(tokens[index] == ','):
            index += 1
            if(not isIdentifier(tokens[index])):
                return index, "expected identifier"
            global_symbol_table[tokens[index]] = [dtype]
            index += 1

            if(tokens[index] == '='):
                index += 1
                index, err, expr_dtype = checkExpr(tokens=tokens, index=index, symbol_table=global_symbol_table)
                if(err):
                    return index, err
                
        if(tokens[index] != ";"):
            return index, "expected ; "
        index += 1
        return index, None
                

    index, err, parameters = checkParametersList(tokens=tokens, index=index)
    print(index,parameters)

    if(err):
        return index, err

    # add function to the symbol table
    global_symbol_table[ident] = [dtype, parameters]

    if(tokens[index] == ";"):
        return index+1, None
    
    local_symbol_table = {}
    local_symbol_table.update(global_symbol_table)
    local_symbol_table.update(parameters)

    index, err = checkScope(tokens, index, local_symbol_table)

    #update
    if(err):
        return index,err

    return index+1, None
    

# Main function

In [350]:
i = 0
while(i < len(tokens)):
    print(i,tokens[i])
    if(tokens[i][0] == '#'):
        i = ignorePreprocessor(tokens=tokens, index=i)
        continue

    if(tokens[i] in datatype):
        # Def includes Decl, Def
        i, err = checkFuncDefOrVarDef(tokens=tokens, index = i)
        if(err):
            print(f"Error: error occured at token {i, tokens[i]}: {err}")
            break
        continue

0 #include
8 int
12 {}
processing 13 with token: int
it is a func declaration or var def
inside checkExpr 16 -
inside checkLval 16 -
inside checkLval 17 52
exited lval with 18, token:,, data_type: int
exited checkLval 18, ,, ldtype: int
exited checkExpr inside var def fun dec 18, ,, err: None
processed func declaration or var def
processing 21 with token: double
it is a func declaration or var def
inside checkExpr 24 first_number
inside checkLval 24 first_number
exited checkLval 25, ;, ldtype: int
exited checkExpr inside var def fun dec 25, ;, err: None
processed func declaration or var def
processing 26 with token: int
it is a func declaration or var def
inside checkExpr 29 a
inside checkLval 29 a
exited checkLval 30, +, ldtype: double
inside checkExpr 31 a
inside checkLval 31 a
exited checkLval 32, ;, ldtype: double
exited checkExpr 32, ;, err: None
checking for semicolon and datatype after rval op, ldtype: double, rdtype: double
ldtype in decimal type
exited checkExpr inside var def

In [351]:
# for i in range(0,len(tokens)):
#     print(i,tokens[i])